In [ ]:
try:
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import cross_val_score,train_test_split,cross_val_predict,KFold,cross_validate
    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
    from bayes_opt import BayesianOptimization
    import matplotlib.pyplot as plt
    import seaborn as sns
    import lightgbm as lgb
    import xgboost as xgb
    from sklearn.manifold import TSNE 
    import time
    from rdkit import Chem
    from rdkit.Chem import AllChem, QED,Draw,MACCSkeys,Descriptors,ChemicalFeatures,DataStructs
    from rdkit.Chem.Draw import rdMolDraw2D
    from rdkit.Chem.Draw import IPythonConsole
    from rdkit import rdBase,DataStructs
    from matplotlib.ticker import AutoMinorLocator,MultipleLocator,FormatStrFormatter,LinearLocator,NullLocator,FixedLocator,IndexLocator,AutoLocator
    import matplotlib as mpl
    from IPython.display import SVG
    from rdkit.ML.Cluster import Butina
    from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions 
except Exception as e:
    print(e)

# Data Processin

In [ ]:
def read_csv(path,size=0.2,random_state=np.random.randint(100000000)):
    df = pd.read_csv(path)
    x,y = df.iloc[:,:-1],df.iloc[:,-1]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=size,random_state=random_state)
    return df,x_train, x_test, y_train, y_test,random_state
df,x_train, x_test, y_train, y_test,random_state = read_csv('../data/Fingerprint/ripk1_finger.csv',random_state=23)

def data_drop(random_state=23,frac=1):
    df_label = df.copy()
    df_label['label'] = df_label['label'].sample(frac=frac,random_state=random_state).reset_index(drop=True)
    return df_label

# Bayesian optimization

In [ ]:
# lightgbm tune
def bayes_objective(eta,max_depth,feature_fraction,num_leaves,lambda_l1,lambda_l2,subsample):
    train = lgb.Dataset(x_train, y_train)
    valid = lgb.Dataset(x_test,y_test)
    params = {'eta': eta,'objective': 'binary', 'metric': 'auc',
    'boosting':'gbdt','lambda_l2': lambda_l2,'max_depth':int(max_depth),
    'feature_fraction':feature_fraction,'lambda_l1':lambda_l1,
    'num_leaves':int(num_leaves),'subsample':subsample,'verbosity':-1} 
    gbm = lgb.train(params, train, num_boost_round=1000, 
           valid_sets=[train,valid],valid_names = ['dtrain','dtest'],early_stopping_rounds=50, verbose_eval=40,
           callbacks=[lgb.early_stopping(50),lgb.record_evaluation({})],feature_name=list(x_train.columns))
    y_pred = gbm.predict(x_test)
    auc = metrics.roc_auc_score(y_test, y_pred)
    return auc
if __name__ == '__main__':
    start = time.time()
    params = {"eta": (0.00005, 1),"num_leaves": (2,100), "feature_fraction": (0.001, 1.0),
        'max_depth':(1,10),'lambda_l1':(0,10),'subsample':(0.001,1.0),'lambda_l2':(0,10)}
    opt = BayesianOptimization(bayes_objective,params)
    opt.maximize(20,20)
    print('\n','\n','best params:',opt.max['params'],'\n','\n','best score:',opt.max['target'],'\t','It takes %s minutes '% ((time.time()-start)/60))
    lgb_params = opt.max['params']

In [ ]:
# xgboost tune
def bayes_objective(eta,max_depth,lambda_,alpha,subsample,min_child_weight,gamma,colsample_bytree):
    train = xgb.DMatrix(x_train, y_train)
    valid = xgb.DMatrix(x_test,y_test)
    params = {'eta': eta,'objective': 'binary:logistic', 'eval_metric': 'auc',
    'booster':'gbtree','lambda': lambda_,'max_depth':int(max_depth),'gamma':gamma,
    'colsample_bytree': colsample_bytree,'min_child_weight':min_child_weight,
    'alpha':alpha,'subsample':subsample,'verbosity':0} 
    xgb_model = xgb.train(params,dtrain = train, num_boost_round=1000,
                                evals=[(train,'dtrain'),(valid, 'eval')] ,
                                early_stopping_rounds=50, verbose_eval=40)
    y_pred = xgb_model.predict(valid)
    auc = metrics.roc_auc_score(y_test, y_pred)
    return auc
if __name__ == '__main__':
    start = time.time()
    params = {"eta": (0.00005, 1),"min_child_weight": (1,5), "colsample_bytree": (0.001, 1.0),
        'max_depth':(1,10),'lambda_':(0,10),'subsample':(0.001,1.0),'alpha':(0,10),'gamma':(0,10)}
    opt = BayesianOptimization(bayes_objective,params)
    opt.maximize(20,20)
    print('\n','\n','best params:',opt.max['params'],'\n','\n','best score:',opt.max['target'],'It takes %s minutes '% ((time.time()-start)/60))
    xgb_params = opt.max['params']

In [ ]:
# rf tune
def bayes_objective(max_depth,min_samples_split,min_samples_leaf,max_features,max_samples,n_estimators):
    params = {'criterion': 'gini','n_estimators':int(n_estimators),'max_depth':int(max_depth),
    'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,'max_samples':max_samples} 
    rfc = RandomForestClassifier(**params)
    rfc_model = rfc.fit(x_train,y_train)
    y_pred = rfc_model.predict_proba(x_test)
    auc = metrics.roc_auc_score(y_test,y_pred[:,1])
    return auc
if __name__ == '__main__':
    start = time.time()
    params = {"min_samples_split": (0.001, 0.9999),"min_samples_leaf": (0.001,0.5), "max_features": (0.001,0.9999),
        'max_depth':(1,20),'max_samples':(0.001,0.9999),'n_estimators':(50,500)}
    opt = BayesianOptimization(bayes_objective,params)
    opt.maximize(50,50)
    print('\n','\n','best params:',opt.max['params'],'\n','\n','best score:',opt.max['target'],'It takes %s minutes '% ((time.time()-start)/60))
    rf_params = opt.max['params']

In [ ]:
# et tune
def bayes_objective(max_depth,min_samples_split,min_samples_leaf,max_features,max_samples,n_estimators):
    params = {'criterion': 'gini','n_estimators':int(n_estimators),'max_depth':int(max_depth),
    'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,'max_samples':max_samples} 
    et = ExtraTreesClassifier(**params)
    et_model = et.fit(x_train,y_train)
    y_pred = et_model.predict_proba(x_test)
    auc = metrics.roc_auc_score(y_test,y_pred[:,1])
    return auc
if __name__ == '__main__':
    start = time.time()
    params = {"min_samples_split": (0.001, 0.9999),"min_samples_leaf": (0.001,0.5), "max_features": (0.001,0.9999),
        'max_depth':(1,20),'max_samples':(0.001,0.9999),'n_estimators':(50,500)}
    opt = BayesianOptimization(bayes_objective,params)
    opt.maximize(50,50)
    print('\n','\n','best params:',opt.max['params'],'\n','\n','best score:',opt.max['target'],'It takes %s minutes '% ((time.time()-start)/60))
    et_params = opt.max['params']

# The best combination of hyperparameters obtained

In [ ]:
lgb_params = {'eta': 0.1,'objective': 'binary','metric':'auc','max_depth':9,
              'feature_fraction':0.7267,'lambda_l1':1,'num_leaves':34,'verbosity':-1}
xgb_params = {'colsample_bytree': 0.8, 'eta': 0.1, 'gamma': 0.1, 'lambda': 1,'max_depth': 6,
              'min_child_weight': 1.0, 'subsample': 0.8,'objective':'binary:logistic','eval_metric': 'auc','verbosity':0}
rf_params = {'max_depth': 10.0, 'max_features': 0.1, 'min_samples_split': 27, 'n_estimators': 72}
et_params = {'max_depth': 19, 'max_features': 0.09966, 'min_samples_split': 59, 'n_estimators': 104} 

# Model Building

In [ ]:
def model(model_type,x_train,y_train,x_test,y_test,random_state=None,**kwargs):
    assert type(model_type)==str,print('string type required')
    if model_type=='lb':
        train = lgb.Dataset(x_train,y_train)
        valid = lgb.Dataset(x_test,y_test)
        lgb_model = lgb.train(kwargs, train_set = train, num_boost_round = 1000, valid_sets=[train,valid],
                                valid_names =['dtrain','dtest'], verbose_eval=30,callbacks=[ 
                                lgb.early_stopping(50),lgb.record_evaluation({})],
                                feature_name=x_train.columns.to_list())
        y_pred = lgb_model.predict(x_test, num_iteration=lgb_model.best_iteration)
        y_train_pred = lgb_model.predict(x_train, num_iteration=lgb_model.best_iteration)
        return lgb_model,y_pred,y_train_pred
    
    elif model_type=='xb':
        train = xgb.DMatrix(x_train, y_train)
        valid = xgb.DMatrix(x_test, y_test)   
        xgb_model = xgb.train(kwargs,dtrain = train, num_boost_round=1000,
                                evals=[(train,'dtrain'),(valid, 'eval')] ,
                                early_stopping_rounds=50, verbose_eval=30)
        y_pred = xgb_model.predict(valid, iteration_range=(0, xgb_model.best_iteration+1))
        y_train_pred = xgb_model.predict(train, iteration_range=(0, xgb_model.best_iteration+1))
        return xgb_model,y_pred,y_train_pred
        
    elif model_type=='rf':
        rfc = RandomForestClassifier(random_state=random_state,**kwargs) # class_weight={0:1,1:1}
        rf_model = rfc.fit(x_train,y_train)
        y_pred = rf_model.predict_proba(x_test)
        y_train_pred = rf_model.predict_proba(x_train)
        print('ok')
        return rf_model,y_pred,y_train_pred
    
    else :
        et = ExtraTreesClassifier(random_state=random_state,**kwargs)
        et_model = et.fit(x_train,y_train)
        y_pred = et_model.predict_proba(x_test)
        y_train_pred = et_model.predict_proba(x_train)
        print('ok')
        return et_model,y_pred,y_train_pred


def model_predict(model:str,ytrue, ypred):
    if model == 'lb' or model == 'xb':
        pred = [1 if y >= 0.5 else 0 for y in ypred]
        auc = metrics.roc_auc_score(y_test, ypred)
        acc = metrics.accuracy_score(y_test,pred)
        precesion = metrics.precision_score(y_test,pred)
        recall = metrics.recall_score(y_test,pred)
        f1 = metrics.f1_score(y_test, pred)
        mcc = metrics.matthews_corrcoef(y_test, pred)
        cm = metrics.confusion_matrix(y_test, pred,labels=[0,1]).astype(np.int32)
        sp = cm[0][0]/(cm[0][0]+cm[0][1])
    
    if model == 'rf' or model == 'et':
        pred = [1 if y >= 0.5 else 0 for y in ypred[:,1]]
        auc = metrics.roc_auc_score(y_test, ypred[:,1])
        acc = metrics.accuracy_score(y_test,pred)
        precesion = metrics.precision_score(y_test,pred)
        recall = metrics.recall_score(y_test,pred)
        f1 = metrics.f1_score(y_test, pred)
        mcc = metrics.matthews_corrcoef(y_test, pred)
        cm = metrics.confusion_matrix(y_test, pred,labels=[0,1]).astype(np.int32)
        sp = cm[0][0]/(cm[0][0]+cm[0][1])
    
    return auc,acc,precesion,recall,f1,mcc,cm,sp

if __name__ == '__main__': 
    lgb_model,lb_y_pred ,lb_y_train_pred= model('lb',x_train,y_train,x_test,y_test,**lgb_params)
    xgb_model,xb_y_pred ,xb_y_train_pred= model('xb',x_train,y_train,x_test,y_test,**xgb_params)
    rf_model,rf_y_pred ,rf_y_train_pred= model('rf',x_train,y_train,x_test,y_test,random_state=123123,**rf_params)
    et_model,et_y_pred ,et_y_train_pred= model('et',x_train,y_train,x_test,y_test,random_state=100,**et_params)
    lb_auc,lb_acc,lb_precesion,lb_recall,lb_f1,lb_mcc,lb_cm,lb_sp = model_predict('lb',y_test,lb_y_pred)
    xb_auc,xb_acc,xb_precesion,xb_recall,xb_f1,xb_mcc,xb_cm,xb_sp = model_predict('xb',y_test,xb_y_pred)
    rf_auc,rf_acc,rf_precesion,rf_recall,rf_f1,rf_mcc,rf_cm,rf_sp = model_predict('rf',y_test,rf_y_pred)
    et_auc,et_acc,et_precesion,et_recall,et_f1,et_mcc,et_cm,et_sp = model_predict('et',y_test,et_y_pred)

# Ten-fold cross-validation

In [ ]:
def cross_valid(model_type,n_splits=10,shuffle=True,random_state=23,**kwargs):
    auc,acc,precesion,recall,f1,cm,mcc,sp = [],[],[],[],[],[],[],[]
    kfold = KFold(n_splits=n_splits,shuffle=shuffle,random_state=random_state)
    print(kwargs)
    if model_type == 'lb':
        for i,j in enumerate(kfold.split(x_train, y_train),1):
            train_index, test_index = j
            this_train_x,this_train_y = x_train.iloc[train_index,:],y_train.values[train_index]   
            this_test_x,this_test_y = x_train.iloc[test_index,:],y_train.values[test_index]  
            train = lgb.Dataset(this_train_x, this_train_y)
            valid = lgb.Dataset(this_test_x,this_test_y)
            lgb_model = lgb.train(kwargs, 
                       train_set = train, 
                       num_boost_round=1000, 
                       valid_sets = [train,valid],
                       valid_names = ['dtrain','dtest'],
                       callbacks = [lgb.early_stopping(50),lgb.record_evaluation({})],
                       verbose_eval = 40,feature_name = list(x_train.columns))
            y_pred = lgb_model.predict(this_test_x, iteration_range=(0, lgb_model.best_iteration+1))
            pred = [1 if y >= 0.5 else 0 for y in y_pred]
            auc.append(metrics.roc_auc_score(this_test_y, y_pred))
            acc.append(metrics.accuracy_score(this_test_y,pred))
            precesion.append(metrics.precision_score(this_test_y,pred))
            recall.append(metrics.recall_score(this_test_y,pred))
            f1.append(metrics.f1_score(this_test_y, pred))
            cm.append(metrics.confusion_matrix(this_test_y, pred,labels=[0,1]))
            mcc.append(metrics.matthews_corrcoef(this_test_y, pred))
            print('*'*30,i,'*'*30)
        sp.extend([cm[i-1][0][0]/(cm[i-1][0][0]+cm[i-1][0][1]) for i in range(kfold.n_splits)])
    
    elif model_type == 'xb':
        for i,j in enumerate(kfold.split(x_train, y_train),1):
            train_index, test_index = j
            this_train_x,this_train_y = x_train.iloc[train_index,:],y_train.values[train_index]   
            this_test_x,this_test_y = x_train.iloc[test_index,:],y_train.values[test_index] 
            train = xgb.DMatrix(this_train_x, this_train_y)
            valid = xgb.DMatrix(this_test_x,this_test_y)
            xgb_model = xgb.train(kwargs, train, num_boost_round=1000,
                             evals=[(train,'dtrain'),(valid, 'eval')], 
                             early_stopping_rounds=50, verbose_eval=40)   
            y_pred = xgb_model.predict(valid, iteration_range=(0, xgb_model.best_iteration+1))
            pred = [1 if y >= 0.5 else 0 for y in y_pred]
            auc.append(metrics.roc_auc_score(this_test_y, y_pred))
            acc.append(metrics.accuracy_score(this_test_y,pred))
            precesion.append(metrics.precision_score(this_test_y,pred))
            recall.append(metrics.recall_score(this_test_y,pred))
            f1.append(metrics.f1_score(this_test_y, pred))
            cm.append(metrics.confusion_matrix(this_test_y, pred,labels=[0,1]))
            mcc.append(metrics.matthews_corrcoef(this_test_y, pred))
            print('*'*30,i,'*'*30)
        sp.extend([cm[i-1][0][0]/(cm[i-1][0][0]+cm[i-1][0][1]) for i in range(kfold.n_splits)])
        
    else :
        for i,j in enumerate(kfold.split(x_train, y_train),1):
            train_index, test_index = j
            this_train_x,this_train_y = x_train.iloc[train_index,:],y_train.values[train_index]
            this_test_x,this_test_y = x_train.iloc[test_index,:], y_train.values[test_index]  
            if model_type == 'rf':
                rfc = RandomForestClassifier(**kwargs)
                rfc_model = rfc.fit(this_train_x,this_train_y)
                y_pred = rfc_model.predict_proba(this_test_x)
            elif model_type == 'et':
                et = ExtraTreesClassifier(**kwargs)
                et_model = et.fit(this_train_x,this_train_y)
                y_pred = et_model.predict_proba(this_test_x)
            pred = [1 if y >= 0.5 else 0 for y in y_pred[:,1]]
            auc.append(metrics.roc_auc_score(this_test_y, y_pred[:,1]))
            acc.append(metrics.accuracy_score(this_test_y,pred))
            precesion.append(metrics.precision_score(this_test_y,pred))
            recall.append(metrics.recall_score(this_test_y,pred))
            f1.append(metrics.f1_score(this_test_y, pred))
            cm.append(metrics.confusion_matrix(this_test_y, pred,labels=[0,1]))
            mcc.append(metrics.matthews_corrcoef(this_test_y, pred))
            print('*'*30,i,'*'*30)
        sp.extend([cm[i-1][0][0]/(cm[i-1][0][0]+cm[i-1][0][1]) for i in range(kfold.n_splits)])
    return auc,acc,precesion,recall,f1,cm,mcc,sp
    
if __name__ == '__main__':
    auc,acc,precesion,recall,f1,cm,mcc,sps = cross_valid('lb',**lgb_params)
#     auc,acc,precesion,recall,f1,cm,mcc,sps = cross_valid('xb',**xgb_params)
#     auc,acc,precesion,recall,f1,cm,mcc,sps = cross_valid('rf',**rf_params)
#     auc,acc,precesion,recall,f1,cm,mcc,sps = cross_valid('et',**et_params)
    print(f'''auc:{np.mean(auc):.4f}\tacc:{np.mean(acc):.4f}\tprecesion:{np.mean(precesion):.4f}
        recall:{np.mean(recall):.4f}\tf1:{np.mean(f1):.4f}\tmcc:{np.mean(mcc):.4f}\tsp:{np.mean(sps):.4f}''')

# Y-randomize

In [ ]:
def y_randomization(model_type):
    assert type(model_type)==str,print('string type required')
    aucs,accs,precesions,recalls,f1s,mccs,cms,sps = [],[],[],[],[],[],[],[]
    for i in range(1,501):
        print('*'*30,i,'*'*30)
        df_label = data_drop(random_state=i)
        if model_type=='lgb':
            lgb_model,y_pred ,y_train_pred= model('lb',x_train,y_train,x_test,y_test,**lgb_params)
            auc,acc,precesion,recall,f1,mcc,cm,sp = model_predict('lb',y_test,y_pred)
            aucs.append(auc)
            accs.append(acc)
            precesions.append(precesion)
            recalls.append(recall)
            f1s.append(f1)
            mccs.append(mcc)
            sps.append(sp)
            cms.append(cm)
        elif model_type=='xgb':
            xgb_model,y_pred ,y_train_pred= model('xb',x_train,y_train,x_test,y_test,**xgb_params)
            auc,acc,precesion,recall,f1,mcc,cm,sp = model_predict('xb',y_test,y_pred)
            aucs.append(auc)
            accs.append(acc)
            precesions.append(precesion)
            recalls.append(recall)
            f1s.append(f1)
            mccs.append(mcc)
            sps.append(sp)
            cms.append(cm)
        elif model_type=='rf':
            rf_model,y_pred ,y_train_pred= model('rf',x_train,y_train,x_test,y_test,random_state=123123,**rf_params)
            auc,acc,precesion,recall,f1,mcc,cm,sp = model_predict('rf',y_test,y_pred)
            aucs.append(auc)
            accs.append(acc)
            precesions.append(precesion)
            recalls.append(recall)
            f1s.append(f1)
            mccs.append(mcc)
            sps.append(sp)
            cms.append(cm)
        else:
            et_model,y_pred ,y_train_pred= model('et',x_train,y_train,x_test,y_test,random_state=100,**et_params)
            auc,acc,precesion,recall,f1,mcc,cm,sp = model_predict('et',y_test,y_pred)
            aucs.append(auc)
            accs.append(acc)
            precesions.append(precesion)
            recalls.append(recall)
            f1s.append(f1)
            mccs.append(mcc)
            sps.append(sp)
            cms.append(cm)
    return aucs,accs,precesions,recalls,f1s,mccs,cms,sps
aucs,accs,precesions,recalls,f1s,mccs,cms,sps = y_randomization('lb')

# Predicting unseen data

In [ ]:
vivo_fp = pd.read_csv('../data/Fingerprint/vivo_all_drop_fp_2048_4.csv')
data_vivo = pd.read_csv('../data/Raw_data/vivo_all_drop.csv')
def unseen_predict():
    all_index = []
    xgbdata = xgb.DMatrix(vivo_fp)
    lgb_predict = np.where(lgb_model.predict(vivo_fp,num_iteration=lgb_model.best_iteration)>=0.5,1,0)
    xgb_predict = np.where(xgb_model.predict(xgbdata, iteration_range=(0, xgb_model.best_iteration+1))>=0.5,1,0)
    rf_predict = np.where(rf_model.predict_proba(vivo_fp)[:,1]>=0.5,1,0)
    et_predict = np.where(et_model.predict_proba(vivo_fp)[:,1]>=0.5,1,0)
    for i,j in enumerate(lgb_predict):
        if j==1:
            all_index.append(i)
    for i,j in enumerate(xgb_predict):
        if j==1:
            all_index.append(i)
    for i,j in enumerate(rf_predict):
        if j==1:
            all_index.append(i)
    for i,j in enumerate(et_predict):
        if j==1:
            all_index.append(i)
    all_index = list(set(all_index))
    return lgb_predict,xgb_predict,rf_predict ,et_predict,all_index
lgb_predict,xgb_predict,rf_predict ,et_predict,all_index = unseen_predict()
data_vivo_new = data_vivo.iloc[all_index]
data_vivo_new.index = range(len(data_vivo_new))

# Filter molecules

In [ ]:
mols = [Chem.MolFromSmiles(smi) for smi in data_vivo_new['smiles']]

MW = []
HA = []
HB = []
ROT = []
PSA = []
qeds = []

for zinc, mol in zip(data_vivo_new['zinc_id'], mols):
    
    link = QED.properties(mol)
    qed = QED.qed(mol)
    MW.append([zinc, link.MW])
    HA.append([zinc, link.HBA])
    HB.append([zinc, link.HBD])
    ROT.append([zinc, link.ROTB])
    PSA.append([zinc, link.ALOGP])
    qeds.append([zinc, qed])

vivo_qed = pd.DataFrame(MW, columns=['zinc_id', 'MW'])
vivo_qed['HBA'] = [zic[1] for zic in HA]
vivo_qed['HBD'] = [zic[1] for zic in HB]
vivo_qed['ROTB'] = [zic[1] for zic in ROT]
vivo_qed['LOGP'] = [zic[1] for zic in PSA]
vivo_qed['QED'] = [zic[1] for zic in qeds]
vivo_qed['smiles'] = data_vivo_new['smiles']

vivo_qed_new = vivo_qed[(vivo_qed["MW"]<= 600) & (vivo_qed["HBA"]<=10) & (vivo_qed["HBD"]<=5) & (vivo_qed["ROTB"]<=10) & (vivo_qed["LOGP"]<=7) & (vivo_qed["LOGP"]>= 0)]
vivo_qed_new.index = range(len(vivo_qed_new))

In [ ]:
import math
import pickle
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import os
import os.path as op
 
_fscores = None
 
 
def readFragmentScores(name='fpscores'):
    import gzip
    global _fscores
    if name == "fpscores":
        name = op.join(os.getcwd(), name)
    data = pickle.load(gzip.open('%s.pkl.gz' % name))
    outDict = {}
    for i in data:
        for j in range(1, len(i)):
            outDict[i[j]] = float(i[0])
    _fscores = outDict
 
 
def numBridgeheadsAndSpiro(mol, ri=None):
    nSpiro = rdMolDescriptors.CalcNumSpiroAtoms(mol)
    nBridgehead = rdMolDescriptors.CalcNumBridgeheadAtoms(mol)
    return nBridgehead, nSpiro
 
 
def calculateScore(m):
    if _fscores is None:
        readFragmentScores()
 
    # fragment score
    fp = rdMolDescriptors.GetMorganFingerprint(m,
                                            2)  # <- 2 is the *radius* of the circular fingerprint
    fps = fp.GetNonzeroElements()
    score1 = 0.
    nf = 0
    for bitId, v in fps.items():
        nf += v
        sfp = bitId
        score1 += _fscores.get(sfp, -4) * v
    score1 /= nf
 
    # features score
    nAtoms = m.GetNumAtoms()
    nChiralCenters = len(Chem.FindMolChiralCenters(m, includeUnassigned=True))
    ri = m.GetRingInfo()
    nBridgeheads, nSpiro = numBridgeheadsAndSpiro(m, ri)
    nMacrocycles = 0
    for x in ri.AtomRings():
        if len(x) > 8:
            nMacrocycles += 1
 
    sizePenalty = nAtoms**1.005 - nAtoms
    stereoPenalty = math.log10(nChiralCenters + 1)
    spiroPenalty = math.log10(nSpiro + 1)
    bridgePenalty = math.log10(nBridgeheads + 1)
    macrocyclePenalty = 0.
    if nMacrocycles > 0:
        macrocyclePenalty = math.log10(2)
 
    score2 = 0. - sizePenalty - stereoPenalty - spiroPenalty - bridgePenalty - macrocyclePenalty
    score3 = 0.
    if nAtoms > len(fps):
        score3 = math.log(float(nAtoms) / len(fps)) * .5
 
    sascore = score1 + score2 + score3
    min = -4.0
    max = 2.5
    sascore = 11. - (sascore - min + 1) / (max - min) * 9.
    # smooth the 10-end
    if sascore > 8.:
        sascore = 8. + math.log(sascore + 1. - 9.)
    if sascore > 10.:
        sascore = 10.0
    elif sascore < 1.:
        sascore = 1.0
 
    return sascore
def my_score(mols:list):
    readFragmentScores("fpscores")
    smiles_s = []
    for i,m in enumerate(mols):
        s = calculateScore(m)
        smiles = Chem.MolToSmiles(m)
        smiles_s.append([i,smiles,s,vivo_qed_new.iloc[i,0]])
    return smiles_s

In [ ]:
smiles=my_score([Chem.MolFromSmiles(i) for i in vivo_qed_new['smiles']])
filter_molecule = []
for i in range(len(smiles)):
    if smiles[i][2] < 4.5:
        filter_molecule.append([smiles[i][1],smiles[i][2],smiles[i][3]])
len(filter_molecule)